# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [165]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import numpy as np

In [166]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [167]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [168]:
df=data.groupby(["CustomerID","ProductName"]).sum()
df.head()

SalesID  ProductID  Quantity
CustomerID ProductName                                                
33         Apricots - Dried                172592        324         1
           Assorted Desserts              5363705        219         1
           Bandage - Flexible Neon        6299165        273         1
           Bar Mix - Pina Colada, 355 Ml  3147536        281         1
           Beans - Kidney, Canned         6507033        300         1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [169]:
table = pd.pivot_table(data, values='Quantity', index=["ProductName"],columns=["CustomerID"], aggfunc=np.sum, fill_value=0)
table.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [170]:
m = pdist(table.T, 'euclidean')
#display(m)
#display(squareform(m))
squareform(pdist(table.T, 'euclidean'))

distances = pd.DataFrame(1/(1 + squareform(pdist(table.T, 'euclidean'))), 
                         index=table.columns, columns=table.columns)

distances.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [171]:
#he seleccionado el CustomerID 33
'''lst=[]
for i in distances.iloc[0]:
    lst.append(i)
lst.sort(reverse=True)
select=lst[1:6]
select'''
similarities = distances.iloc[0].sort_values(ascending=False)[1:6]
similarities

CustomerID
264     0.087047
3535    0.087047
3317    0.087047
2503    0.085983
3305    0.085638
Name: 33, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [172]:
lst=list(similarities.index)
lst
step_5=df.loc[lst,]
step_5.head()

SalesID  ProductID  Quantity
CustomerID ProductName                                             
264        Apricots - Halves           3730301        278         1
           Apricots Fresh              5035449        158         1
           Bacardi Breezer - Tropical   756802         49         1
           Bagel - Plain               6728381        262         1
           Banana - Leaves             6298459        271         1

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [173]:
step_6=step_5.groupby(["ProductName"])["Quantity"].agg("sum").sort_values( ascending=False)
step_6.head()

ProductName
Butter - Unsalted                3
Soup - Campbells Bean Medley     3
Towels - Paper / Kraft           3
Wine - Ej Gallo Sierra Valley    3
Pepper - Black, Whole            3
Name: Quantity, dtype: int64

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [174]:
df_merge=table.merge(step_6, on="ProductName")
df_merge_33=df_merge.iloc[:,[0,-1]]
filter1 =  df_merge_33[df_merge_33.iloc[:,0]==0]
a=filter1["Quantity"].sort_values(ascending=False).head(5)


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [175]:
ids=data["CustomerID"].value_counts()
list_ID=list(ids.index)

In [176]:
dict_similarities={}
dict_index={}
dict_aggregate={}
dict_quantities_left={}
for i in list_ID:
    similarities = distances.loc[:,i].sort_values(ascending=False)[1:6]
    dict_similarities[i]=similarities
    
    lst=list(similarities.index)
    step_5=df.loc[lst,]
    dict_index[i]=step_5.head()
    
    step_6=step_5.groupby(["ProductName"])["Quantity"].agg("sum").sort_values( ascending=False)
    dict_aggregate[i]=step_6.head()
    
    df_merge=table.merge(step_6, on="ProductName")
    df_merge_33=df_merge.loc[:,[i,"Quantity"]]
    filter1 =  df_merge_33[df_merge_33.iloc[:,0]==0]
    a=filter1["Quantity"].sort_values(ascending=False).head(5)
    dict_quantities_left[i]=list(a.index)


##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [177]:
product1=[e[0] for e in list(dict_quantities_left.values())]
product2=[e[1] for e in list(dict_quantities_left.values())]
product3=[e[2] for e in list(dict_quantities_left.values())]
product4=[e[3] for e in list(dict_quantities_left.values())]
product5=[e[4] for e in list(dict_quantities_left.values())]
key=list(dict_quantities_left.keys())

dict_final={"CustomerID":key, "Poduct1":product1, "Product2": product2, "Product3": product3, "Product4": product4, "Product5": product5}

dataFrame=pd.DataFrame.from_dict(dict_final)
dataFrame.head()


,CustomerID,Poduct1,Product2,Product3,Product4,Product5
0,33759,Shrimp - 31/40,"Coconut - Shredded, Sweet",Bread - French Baquette,Lambcasing,Pecan Raisin - Tarts
1,60862,Onions - Vidalia,Chips Potato Salt Vinegar 43g,Sugar - Fine,"Pepsi - Diet, 355 Ml",Wine - Vineland Estate Semi - Dry
2,29287,Beer - Sleemans Cream Ale,Bread - Italian Corn Meal Poly,Table Cloth 54x72 White,Cookie - Dough Variety,Cheese - Wine
3,8711,Soup - Campbells Tomato Ravioli,Sauce - Hollandaise,Bread - Raisin Walnut Oval,Water - Aquafina Vitamin,Veal - Eye Of Round
4,63086,Sausage - Liver,Beef - Top Sirloin - Aaa,Sardines,Oil - Shortening - All - Purpose,"Wine - Red, Colio Cabernet"


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [178]:
m = pdist(table.T, 'euclidean')
#display(m)
#display(squareform(m))
squareform(pdist(table.T, 'euclidean'))

distances = pd.DataFrame(1/(1 + squareform(pdist(table.T, 'correlation'))), 
                         index=table.columns, columns=table.columns)

distances.head()

ids=data["CustomerID"].value_counts()
list_ID=list(ids.index)

dict_similarities={}
dict_index={}
dict_aggregate={}
dict_quantities_left={}
for i in list_ID:
    similarities = distances.loc[:,i].sort_values(ascending=False)[1:6]
    dict_similarities[i]=similarities
    
    lst=list(similarities.index)
    step_5=df.loc[lst,]
    dict_index[i]=step_5.head()
    
    step_6=step_5.groupby(["ProductName"])["Quantity"].agg("sum").sort_values( ascending=False)
    dict_aggregate[i]=step_6.head()
    
    df_merge=table.merge(step_6, on="ProductName")
    df_merge_33=df_merge.loc[:,[i,"Quantity"]]
    filter1 =  df_merge_33[df_merge_33.iloc[:,0]==0]
    a=filter1["Quantity"].sort_values(ascending=False).head(5)
    dict_quantities_left[i]=list(a.index)

product1=[e[0] for e in list(dict_quantities_left.values())]
product2=[e[1] for e in list(dict_quantities_left.values())]
product3=[e[2] for e in list(dict_quantities_left.values())]
product4=[e[3] for e in list(dict_quantities_left.values())]
product5=[e[4] for e in list(dict_quantities_left.values())]
key=list(dict_quantities_left.keys())

dict_final={"CustomerID":key, "Poduct1":product1, "Product2": product2, "Product3": product3, "Product4": product4, "Product5": product5}

dataFrame=pd.DataFrame.from_dict(dict_final)
dataFrame.head()


,CustomerID,Poduct1,Product2,Product3,Product4,Product5
0,33759,Beans - Wax,Cake - Mini Cheesecake,Pail For Lid 1537,"Salsify, Organic",Broom - Corn
1,60862,Snapple Lemon Tea,Foam Cup 6 Oz,Ketchup - Tomato,Cinnamon Buns Sticky,Mustard - Seed
2,29287,Dc Hikiage Hira Huba,"Beans - Kidney, Canned",Oil - Shortening - All - Purpose,Cheese - Bocconcini,Assorted Desserts
3,8711,Olive - Spread Tapenade,Table Cloth 81x81 White,Veal - Sweetbread,"Chocolate - Semi Sweet, Calets",Yogurt - French Vanilla
4,63086,Mussels - Frozen,Tea - Jasmin Green,Bouq All Italian - Primerba,Muffin - Zero Transfat,Wine - Alsace Gewurztraminer
